# Regression with Keras

In [1]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## Generate a dataset

Generate the dataset. We work with one-dimensional feature vectors, with feature values generated uniformly in $[-5,5]$. The target values will be normally distributed (mean $\mu = 0$, standard deviation $\sigma=50$) around a cubic function of the feature values, with the law

$$
y = F(x) = x^3 - \frac{1}{2}\,x^2 + 20\,x
$$

In [2]:
X = np.random.uniform(-5.0, 5.0, size=(1000,1))
Y = np.random.normal(X**3 - 5.0*X**2 + 20.0*X, 50.0)

In [3]:
trace = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    )
)

data = [trace]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    )
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## Build the neural network with Keras

Keras offers an high-level interface to the underlying backend (TensorFlow in this case). A neural network is Keras is defined as a `model` object to which we can add layers to build the graph. Here we work with a feed-forward neural network built with fully connected `Dense` layers.

For general advice on nonlinear regression using Keras, see: https://github.com/keras-team/keras/issues/1874

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import mae

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


For the first layer of the graph the `input_shape` keyword argument must be specified. This is a tuple of integers and does __not__ include the sample axis. Keras expects the input to have the form (n_samples, \[features\]), and `input_shape` must correspond to the shape of the features tensor.

__Example:__ if we have 50 samples, each of which is an array (tensor of rank 1 in TensorFlow terms) with 10 components, we must specify `input_shape=(10,)`.

The dimension of the last layer decides the shape of the output. Here we start with 1-dimensional inputs and we end up with 1-dimensional outputs, as we have a function $F: \mathbb{R}\to\mathbb{R}$. The hidden (intermediate) layers, on the other hand, are wider.

In [94]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.add(Dense(50, activation='tanh'))
model.add(Dense(5))
model.add(Dense(1))

A common issue when doing regression with neural network is the __exploding gradients problem__, where the loss function diverges during the training phase. In this case, it was sufficient to switch from stochastic gradient descent to an Adam optimizer to solve that. The Adam optimizer is a generalization of stochastic gradient descent that exploits an adaptive learning rate.

For information about the exploding gradients problem, see:
- https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
- http://neuralnetworksanddeeplearning.com/chap5.html

For information about the Adam optimizer, see: https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

To initialize the neural network we just call a model's `compile()` method. This include the initialization of all the weights of the layers.

In [95]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

A model's `summary()` method gives an overview of the structure of the graph. In particular, from this we can see the number of parameters the neural network possesses.

In [96]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 1)                 2         
_________________________________________________________________
dense_64 (Dense)             (None, 50)                100       
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 255       
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 363
Trainable params: 363
Non-trainable params: 0
_________________________________________________________________


## Fit the neural network to the data

Keras provides an interface for training and predicting completely analogous to that of a SKLearn model. Among the options, we can specify the `epochs` parameters.

__Epochs:__ number of passages over the whole dataset the neural network will do in the training phase.

Since the dataset is small we are not dividing it in batches, meaning that each gradient descent (or Adam) step uses the exact gradient descent. This also implies that there are no stochastic features in the path followed in parameters space.

In [97]:
model.fit(X, Y, epochs=500)

Epoch 1/500
1000/1000 [==============================] - 1s 946us/step - loss: 16326.7354 - mean_squared_error: 16326.7354
Epoch 2/500
1000/1000 [==============================] - 0s 66us/step - loss: 15976.0167 - mean_squared_error: 15976.0167
Epoch 3/500
1000/1000 [==============================] - 0s 82us/step - loss: 15458.6437 - mean_squared_error: 15458.6437
Epoch 4/500
1000/1000 [==============================] - 0s 67us/step - loss: 14663.9355 - mean_squared_error: 14663.9355
Epoch 5/500
1000/1000 [==============================] - 0s 63us/step - loss: 13561.8806 - mean_squared_error: 13561.8806
Epoch 6/500
1000/1000 [==============================] - 0s 76us/step - loss: 12236.1125 - mean_squared_error: 12236.1125
Epoch 7/500
1000/1000 [==============================] - 0s 69us/step - loss: 10887.2067 - mean_squared_error: 10887.2067
Epoch 8/500
1000/1000 [==============================] - 0s 58us/step - loss: 9596.0134 - mean_squared_error: 9596.0134
Epoch 9/500
1000/1000 [==

1000/1000 [==============================] - 0s 83us/step - loss: 2484.8678 - mean_squared_error: 2484.8678
Epoch 136/500
1000/1000 [==============================] - 0s 92us/step - loss: 2485.6552 - mean_squared_error: 2485.6552
Epoch 137/500
1000/1000 [==============================] - 0s 78us/step - loss: 2481.1887 - mean_squared_error: 2481.1887
Epoch 138/500
1000/1000 [==============================] - 0s 76us/step - loss: 2480.9238 - mean_squared_error: 2480.9238
Epoch 139/500
1000/1000 [==============================] - 0s 77us/step - loss: 2478.5862 - mean_squared_error: 2478.5862
Epoch 140/500
1000/1000 [==============================] - 0s 74us/step - loss: 2479.4025 - mean_squared_error: 2479.4025
Epoch 141/500
1000/1000 [==============================] - 0s 86us/step - loss: 2474.3442 - mean_squared_error: 2474.3442
Epoch 142/500
1000/1000 [==============================] - 0s 90us/step - loss: 2489.2017 - mean_squared_error: 2489.2017
Epoch 143/500
1000/1000 [=============

1000/1000 [==============================] - 0s 64us/step - loss: 2463.7207 - mean_squared_error: 2463.7207
Epoch 270/500
1000/1000 [==============================] - 0s 66us/step - loss: 2463.7436 - mean_squared_error: 2463.7436
Epoch 271/500
1000/1000 [==============================] - 0s 76us/step - loss: 2468.2221 - mean_squared_error: 2468.2221
Epoch 272/500
1000/1000 [==============================] - 0s 70us/step - loss: 2472.4504 - mean_squared_error: 2472.4504
Epoch 273/500
1000/1000 [==============================] - 0s 53us/step - loss: 2464.0751 - mean_squared_error: 2464.0751
Epoch 274/500
1000/1000 [==============================] - 0s 76us/step - loss: 2462.4827 - mean_squared_error: 2462.4827
Epoch 275/500
1000/1000 [==============================] - 0s 81us/step - loss: 2460.3098 - mean_squared_error: 2460.3098
Epoch 276/500
1000/1000 [==============================] - 0s 65us/step - loss: 2463.4075 - mean_squared_error: 2463.4075
Epoch 277/500
1000/1000 [=============

1000/1000 [==============================] - 0s 62us/step - loss: 2460.5126 - mean_squared_error: 2460.5126
Epoch 404/500
1000/1000 [==============================] - 0s 84us/step - loss: 2469.6306 - mean_squared_error: 2469.6306
Epoch 405/500
1000/1000 [==============================] - 0s 72us/step - loss: 2471.6544 - mean_squared_error: 2471.6544
Epoch 406/500
1000/1000 [==============================] - 0s 67us/step - loss: 2458.7277 - mean_squared_error: 2458.7277
Epoch 407/500
1000/1000 [==============================] - 0s 66us/step - loss: 2458.9766 - mean_squared_error: 2458.9766
Epoch 408/500
1000/1000 [==============================] - 0s 72us/step - loss: 2463.2042 - mean_squared_error: 2463.2042
Epoch 409/500
1000/1000 [==============================] - 0s 59us/step - loss: 2459.0076 - mean_squared_error: 2459.0076
Epoch 410/500
1000/1000 [==============================] - 0s 69us/step - loss: 2463.0855 - mean_squared_error: 2463.0855
Epoch 411/500
1000/1000 [=============

## Generate a new dataset and get predictions

To check whether the neural network did a good job, we can make prediction over a (1-dimensional) grid of values and compare it with the true value of the function to fit on the same grid.

In [98]:
X_pred = np.linspace(-10.0, 10.0, 1000).reshape((1000,1))
Y_pred = model.predict(X_pred)

In [99]:
def f_to_fit(x):
    return x**3 - 5.0*x**2 + 20.0*x

In [100]:
trace_data = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'data'
)

trace_pred = go.Scatter(
    x = X_pred[:,0],
    y = Y_pred[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'predictions'
)

trace_true = go.Scatter(
    x = X_pred[:,0],
    y = f_to_fit(X_pred)[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'true values'
)

data = [trace_data, trace_pred, trace_true]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    ),
    hovermode = 'closest'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)

## More experiments

We can tweak the topology of the graph and the number of epochs to see how the results vary. The best generalization seems to be the one using wider layers with `tanh` activation function followed by narrower linear layers.

In [129]:
model_2 = Sequential()
model_2.add(Dense(1, input_shape=(1,)))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(10, activation='tanh'))
model_2.add(Dense(5))
model_2.add(Dense(1))

model_2.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mse']
)

model_2.fit(X, Y, epochs=500)

Epoch 1/500
1000/1000 [==============================] - 2s 2ms/step - loss: 16447.5110 - mean_squared_error: 16447.5110
Epoch 2/500
1000/1000 [==============================] - 0s 149us/step - loss: 16207.6448 - mean_squared_error: 16207.6448
Epoch 3/500
1000/1000 [==============================] - 0s 125us/step - loss: 15896.3370 - mean_squared_error: 15896.3370
Epoch 4/500
1000/1000 [==============================] - 0s 124us/step - loss: 15574.8035 - mean_squared_error: 15574.8035
Epoch 5/500
1000/1000 [==============================] - 0s 123us/step - loss: 15252.9155 - mean_squared_error: 15252.9155
Epoch 6/500
1000/1000 [==============================] - 0s 141us/step - loss: 14927.9141 - mean_squared_error: 14927.9141
Epoch 7/500
1000/1000 [==============================] - 0s 125us/step - loss: 14586.0333 - mean_squared_error: 14586.0333
Epoch 8/500
1000/1000 [==============================] - 0s 135us/step - loss: 14238.9975 - mean_squared_error: 14238.9975
Epoch 9/500
1000/1

1000/1000 [==============================] - 0s 219us/step - loss: 2870.4851 - mean_squared_error: 2870.4851
Epoch 68/500
1000/1000 [==============================] - 0s 204us/step - loss: 2810.6544 - mean_squared_error: 2810.6544
Epoch 69/500
1000/1000 [==============================] - 0s 177us/step - loss: 2768.7043 - mean_squared_error: 2768.7043
Epoch 70/500
1000/1000 [==============================] - 0s 199us/step - loss: 2751.2099 - mean_squared_error: 2751.2099
Epoch 71/500
1000/1000 [==============================] - 0s 183us/step - loss: 2736.0292 - mean_squared_error: 2736.0292
Epoch 72/500
1000/1000 [==============================] - 0s 181us/step - loss: 2711.2943 - mean_squared_error: 2711.2943
Epoch 73/500
1000/1000 [==============================] - 0s 187us/step - loss: 2711.4420 - mean_squared_error: 2711.4420
Epoch 74/500
1000/1000 [==============================] - 0s 144us/step - loss: 2712.2468 - mean_squared_error: 2712.2468
Epoch 75/500
1000/1000 [=============

Epoch 134/500
1000/1000 [==============================] - 0s 200us/step - loss: 2482.0686 - mean_squared_error: 2482.0686
Epoch 135/500
1000/1000 [==============================] - 0s 180us/step - loss: 2481.7880 - mean_squared_error: 2481.7880
Epoch 136/500
1000/1000 [==============================] - ETA: 0s - loss: 2515.0157 - mean_squared_error: 2515.01 - 0s 197us/step - loss: 2456.5951 - mean_squared_error: 2456.5951
Epoch 137/500
1000/1000 [==============================] - 0s 258us/step - loss: 2467.1359 - mean_squared_error: 2467.1359
Epoch 138/500
1000/1000 [==============================] - 0s 199us/step - loss: 2466.1656 - mean_squared_error: 2466.1656
Epoch 139/500
1000/1000 [==============================] - 0s 172us/step - loss: 2498.0525 - mean_squared_error: 2498.0525
Epoch 140/500
1000/1000 [==============================] - 0s 173us/step - loss: 2448.5277 - mean_squared_error: 2448.5277
Epoch 141/500
1000/1000 [==============================] - 0s 269us/step - loss: 

1000/1000 [==============================] - 0s 179us/step - loss: 2457.2757 - mean_squared_error: 2457.2757
Epoch 200/500
1000/1000 [==============================] - 0s 211us/step - loss: 2455.7919 - mean_squared_error: 2455.7919
Epoch 201/500
1000/1000 [==============================] - 0s 146us/step - loss: 2456.4297 - mean_squared_error: 2456.4297
Epoch 202/500
1000/1000 [==============================] - 0s 156us/step - loss: 2456.1771 - mean_squared_error: 2456.1771
Epoch 203/500
1000/1000 [==============================] - 0s 126us/step - loss: 2460.9602 - mean_squared_error: 2460.9602
Epoch 204/500
1000/1000 [==============================] - 0s 120us/step - loss: 2473.2925 - mean_squared_error: 2473.2925
Epoch 205/500
1000/1000 [==============================] - 0s 116us/step - loss: 2445.1232 - mean_squared_error: 2445.1232
Epoch 206/500
1000/1000 [==============================] - 0s 114us/step - loss: 2464.5385 - mean_squared_error: 2464.5385
Epoch 207/500
1000/1000 [=====

1000/1000 [==============================] - 0s 152us/step - loss: 2476.9705 - mean_squared_error: 2476.9705
Epoch 265/500
1000/1000 [==============================] - 0s 167us/step - loss: 2464.0489 - mean_squared_error: 2464.0489
Epoch 266/500
1000/1000 [==============================] - 0s 258us/step - loss: 2459.4813 - mean_squared_error: 2459.4813
Epoch 267/500
1000/1000 [==============================] - 0s 171us/step - loss: 2481.7426 - mean_squared_error: 2481.7426
Epoch 268/500
1000/1000 [==============================] - 0s 167us/step - loss: 2441.8321 - mean_squared_error: 2441.8321
Epoch 269/500
1000/1000 [==============================] - 0s 160us/step - loss: 2466.9644 - mean_squared_error: 2466.9644
Epoch 270/500
1000/1000 [==============================] - 0s 171us/step - loss: 2472.3618 - mean_squared_error: 2472.3618
Epoch 271/500
1000/1000 [==============================] - 0s 208us/step - loss: 2459.9842 - mean_squared_error: 2459.9842
Epoch 272/500
1000/1000 [=====

1000/1000 [==============================] - 0s 112us/step - loss: 2500.6104 - mean_squared_error: 2500.6104
Epoch 330/500
1000/1000 [==============================] - 0s 147us/step - loss: 2445.2288 - mean_squared_error: 2445.2288
Epoch 331/500
1000/1000 [==============================] - 0s 208us/step - loss: 2469.5632 - mean_squared_error: 2469.5632
Epoch 332/500
1000/1000 [==============================] - 0s 118us/step - loss: 2473.4894 - mean_squared_error: 2473.4894
Epoch 333/500
1000/1000 [==============================] - 0s 121us/step - loss: 2476.0797 - mean_squared_error: 2476.0797
Epoch 334/500
1000/1000 [==============================] - 0s 123us/step - loss: 2486.7522 - mean_squared_error: 2486.7522
Epoch 335/500
1000/1000 [==============================] - 0s 128us/step - loss: 2455.8773 - mean_squared_error: 2455.8773
Epoch 336/500
1000/1000 [==============================] - 0s 139us/step - loss: 2460.1922 - mean_squared_error: 2460.1922
Epoch 337/500
1000/1000 [=====

1000/1000 [==============================] - 0s 191us/step - loss: 2457.1209 - mean_squared_error: 2457.1209
Epoch 396/500
1000/1000 [==============================] - 0s 179us/step - loss: 2462.2124 - mean_squared_error: 2462.21240s - loss: 2467.3224 - mean_squared_error: 2467.32
Epoch 397/500
1000/1000 [==============================] - 0s 153us/step - loss: 2464.2584 - mean_squared_error: 2464.2584
Epoch 398/500
1000/1000 [==============================] - 0s 110us/step - loss: 2434.3747 - mean_squared_error: 2434.3747
Epoch 399/500
1000/1000 [==============================] - 0s 158us/step - loss: 2466.8063 - mean_squared_error: 2466.8063
Epoch 400/500
1000/1000 [==============================] - 0s 134us/step - loss: 2454.6190 - mean_squared_error: 2454.6190
Epoch 401/500
1000/1000 [==============================] - 0s 188us/step - loss: 2452.1449 - mean_squared_error: 2452.1449
Epoch 402/500
1000/1000 [==============================] - 0s 115us/step - loss: 2447.0414 - mean_squar

1000/1000 [==============================] - 0s 132us/step - loss: 2438.4773 - mean_squared_error: 2438.4773
Epoch 462/500
1000/1000 [==============================] - 0s 108us/step - loss: 2454.9043 - mean_squared_error: 2454.9043
Epoch 463/500
1000/1000 [==============================] - 0s 116us/step - loss: 2443.1946 - mean_squared_error: 2443.1946
Epoch 464/500
1000/1000 [==============================] - 0s 118us/step - loss: 2481.3903 - mean_squared_error: 2481.3903
Epoch 465/500
1000/1000 [==============================] - 0s 246us/step - loss: 2447.1848 - mean_squared_error: 2447.1848
Epoch 466/500
1000/1000 [==============================] - 0s 171us/step - loss: 2457.3711 - mean_squared_error: 2457.3711
Epoch 467/500
1000/1000 [==============================] - 0s 120us/step - loss: 2446.6828 - mean_squared_error: 2446.6828
Epoch 468/500
1000/1000 [==============================] - 0s 88us/step - loss: 2454.0371 - mean_squared_error: 2454.0371
Epoch 469/500
1000/1000 [======

In [130]:
Y_pred = model_2.predict(X_pred)

trace_data = go.Scatter(
    x = X[:,0],
    y = Y[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'data'
)

trace_pred = go.Scatter(
    x = X_pred[:,0],
    y = Y_pred[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'predictions'
)

trace_true = go.Scatter(
    x = X_pred[:,0],
    y = f_to_fit(X_pred)[:,0],
    mode = 'markers',
    marker = dict(
        size = 3
    ),
    name = 'true values'
)

data = [trace_data, trace_pred, trace_true]

layout = go.Layout(
    xaxis = dict(
        title = 'x'
    ),
    yaxis = dict(
        title = 'y'
    ),
    hovermode = 'closest'
)

fig = go.Figure(data=data, layout=layout)

iplot(fig)